In [ ]:
from vampyr import vampyr3d as vp
from orbital4c import orbital as orb
from orbital4c import complex_fcn as cf
from orbital4c import operators as opr
import numpy as np
from scipy.linalg import eig, inv
import importlib
importlib.reload(orb)
from scipy.special import legendre, laguerre, erf, gamma
from scipy.special import gamma
from scipy.constants import hbar
import numpy.linalg as LA


c = 137   # NOT A GOOD WAY. MUST BE FIXED!!!
alpha = 1/c
k = -1
l = 0
n = 1
m = 0.5
Z = 2

mra = vp.MultiResolutionAnalysis(box=[-20,20], order=7)
prec = 1.0e-4
origin = [0.1, 0.2, 0.3]  # origin moved to avoid placing the nuclar charge on a node

In [ ]:
a_coeff = 3.0
b_coeff = np.sqrt(a_coeff/np.pi)**3
gauss = vp.GaussFunc(b_coeff, a_coeff, origin)
gauss_tree = vp.FunctionTree(mra)
vp.advanced.build_grid(out=gauss_tree, inp=gauss)
vp.advanced.project(prec=prec, out=gauss_tree, inp=gauss)
gauss_tree.normalize()

In [ ]:
orb.orbital4c.mra = mra
cf.complex_fcn.mra = mra
complexfc = cf.complex_fcn()
complexfc.copy_fcns(real=gauss_tree)

In [ ]:
spinorb1 = orb.orbital4c()
spinorb1.copy_components(La=complexfc)
spinorb1.init_small_components(prec/10)
spinorb1.normalize()
spinorb1.crop(prec)

spinorb2 = orb.orbital4c()
spinorb2.copy_components(Lb=complexfc)
spinorb2.init_small_components(prec/10)
spinorb2.normalize()
spinorb2.crop(prec)

spinorb_vec = np.array([spinorb1,spinorb2])

In [5]:
def u(r):
    u = erf(r)/r + (1/(3*np.sqrt(np.pi)))*(np.exp(-(r**2)) + 16*np.exp(-4*r**2))
    #erf(r) is an error function that is supposed to stop the potential well from going to inf.
    #if i remember correctly
    return u

def V(x):
    r = np.sqrt(x[0]**2 + x[1]**2 + x[2]**2)
#    c = 0.0435
    c = 0.000435 # ten times tighter nuclear potential
    f_bar = u(r/c)/c
    return f_bar

Peps = vp.ScalingProjector(mra,prec)
f = lambda x: V([x[0]-origin[0],x[1]-origin[1],x[2]-origin[2]])

print(f([0.5,0.5,0.5]))
V_tree = Z*Peps(f)

1.8569533817705184


In [ ]:
# Working on Helium with Coulomb direct (CJ) & exchange (CK)
GJ_n = opr.GauntDirect(mra, spinorb_vec, prec)
GK_n = opr.GauntExchange(mra, spinorb_vec, prec)

In [ ]:
GJPhi_vec = GJ_n(spinorb_vec)
print(GJPhi_vec)
GKPhi_vec = GK_n(spinorb_vec)
print(GKPhi_vec)


In [6]:
# Working on Helium with Coulomb direct (CJ) & exchange (CK)
J_n = opr.CouloumbOperator(mra, spinorb_vec, prec)
K_n = opr.ExchangeOperator(mra, spinorb_vec, prec)


def calc_overlap(Bra,Ket):
     S_m = np.empty((len(Bra), len(Ket)))
     for i in range(len(Bra)):
        for j in range(len(Ket)):            
            S_m[i, j] = (Bra[i].dot(Ket[j])[0])
     return S_m


#SCF parameters
orbital_error = np.ones(len(spinorb_vec))

# SCF loop
#while (max(orbital_error)) > prec:
for idx in range(1):

    print("step 1")
    hdPhi_vec = [] 
    vPhi_vec = [] 
    orb_energy_vec = []
    tmp2 = []

    print("step 2")
    for i in range(spinorb_vec.shape[0]):
        hdPhi = (orb.apply_dirac_hamiltonian(spinorb_vec[i], prec, 0.0))
        vPhi = (orb.apply_potential(-1.0, V_tree, spinorb_vec[i], prec))
        hdPhi.crop(prec)
        vPhi.crop(prec)
        hdPhi_vec.append(hdPhi)
        vPhi_vec.append(vPhi)
    hdPhi_vec = np.array(hdPhi_vec)
    vPhi_vec = np.array(vPhi_vec)
    
    print("step 3")
    JPhi_vec = J_n(spinorb_vec)
    KPhi_vec = K_n(spinorb_vec)
    
    print("step 4")
    # Definition of full 4c hamitoninan
    add_psi_vec = hdPhi_vec + vPhi_vec + JPhi_vec - KPhi_vec
    
    print("step 5")
    #Calculating of Fij with i /= j  and Lamda matrix
    Fij_matrix = calc_overlap(spinorb_vec, add_psi_vec)
    Lambda_n = np.diag(np.diag(Fij_matrix))
    

    print("step 6")
    orb_energy_vec = np.diag(Fij_matrix)
    print('Orb_Energy', orb_energy_vec)

    print("step 7")
    V_mat = calc_overlap(spinorb_vec, vPhi_vec)
    J_mat = calc_overlap(spinorb_vec, JPhi_vec)
    K_mat = calc_overlap(spinorb_vec, KPhi_vec)

    print("step 8")
    E_orb  =  Fij_matrix.trace()
    E_en   =  V_mat.trace()
    E_coul =  0.5*J_mat.trace()
    E_ex   = -0.5*K_mat.trace()
    E_tot  = E_orb - E_coul - E_ex
    print("E_total(2CJ-CK)", E_tot)

    print("step 9")
    # Calculation of necessary potential contributions to Hellmotz
    tmp = add_psi_vec + (Lambda_n - Fij_matrix) @ spinorb_vec
    
    print("step 10")
    for i in range(0, tmp.shape[0]): 
        tmp2.append(orb.apply_helmholtz(tmp[i], orb_energy_vec[i], c, prec))
    tmp2 =  np.array(tmp2)
    
    print("step 11")
    new_orbital_vec = []
    for i in range(spinorb_vec.shape[0]):
        new_orbital_vec.append(orb.apply_dirac_hamiltonian(tmp2[i], prec, orb_energy_vec[i]))
        new_orbital_vec[i].normalize()    
    new_orbital_vec = np.array(new_orbital_vec)

    print("step 12")
    delta_psi = new_orbital_vec - spinorb_vec
    orbital_error = np.array([delta_psi.norm() for delta_psi in delta_psi])

    print("step 13")
    # Compute overlap matrices
    S_tilde = calc_overlap(new_orbital_vec, new_orbital_vec)
    
    print("step 14")
    # Löwdin orthonormalization S^{-1/2} = U * Sigma^{-1/2} * U^T
    sigma, U = LA.eig(S_tilde)
    Sm5 = U @ np.diag(sigma**(-0.5)) @ U.transpose()
    spinorb_vec = Sm5 @  new_orbital_vec

    print("step 15")
    for orbital in new_orbital_vec:
        orbital.crop(prec)
    
hdPhi_vec = [] 
vPhi_vec = [] 
orb_energy_vec = []
tmp2 = []


for i in range(spinorb_vec.shape[0]):
    hdPhi_vec.append(orb.apply_dirac_hamiltonian(spinorb_vec[i], prec, 0.0))    
    vPhi_vec.append(orb.apply_potential(-1.0, V_tree, spinorb_vec[i], prec))   
hdPhi_vec = np.array(hdPhi_vec)
vPhi_vec = np.array(vPhi_vec)

JPhi_vec = J_n(spinorb_vec)
KPhi_vec = K_n(spinorb_vec)

# Definition of full 4c hamitoninan
add_psi_vec = hdPhi_vec + vPhi_vec + JPhi_vec - KPhi_vec

#Calculating of Fij with i /= j  and Lamda matrix
Fij_matrix = calc_overlap(spinorb_vec, add_psi_vec)
Lambda_n = np.diag(np.diag(Fij_matrix))


orb_energy_vec = np.diag(Fij_matrix)
print('Orb_Energy', orb_energy_vec)

V_mat = calc_overlap(spinorb_vec, vPhi_vec)
J_mat = calc_overlap(spinorb_vec, JPhi_vec)
K_mat = calc_overlap(spinorb_vec, KPhi_vec)

E_orb  =  Fij_matrix.trace()
E_en   =  V_mat.trace()
E_coul =  0.5*J_mat.trace()
E_ex   = -0.5*K_mat.trace()
E_tot  = E_orb - E_coul - E_ex
print("E_total(2CJ-CK)", E_tot)

real non zero
imag non zero
real non zero
imag non zero
real non zero
real non zero
imag non zero
imag non zero
step 1
step 2
step 3
real non zero
imag non zero
real non zero
imag non zero
real non zero
imag non zero
real non zero
imag non zero
real non zero
real non zero
imag non zero
imag non zero
real non zero
real non zero
imag non zero
imag non zero
step 4
step 5
step 6
Orb_Energy [18768.4066986 18768.4066986]
step 7
step 8
E_total(2CJ-CK) 37535.72339312261
step 9
step 10
mu 1.0893043853780873
mu 1.0893043853780873
mu 1.0893043853780873
mu 1.0893043853780873
mu 1.0893043853824602
mu 1.0893043853824602
mu 1.0893043853824602
mu 1.0893043853824602
step 11
step 12
step 13
step 14
step 15
real non zero
imag non zero
real non zero
imag non zero
real non zero
imag non zero
real non zero
imag non zero
real non zero
imag non zero
real non zero
imag non zero
real non zero
imag non zero
real non zero
imag non zero
real non zero
imag non zero
real non zero
imag non zero
real non zero
imag non

In [ ]:
# Working on Helium with Dirac-Coulomb-Gaunt Hamiltonian 

J_n = opr.CouloumbOperator(mra, spinorb_vec, prec)
K_n = opr.ExchangeOperator(mra, spinorb_vec, prec)


def calc_overlap(Bra,Ket):
     S_m = np.empty((len(Bra), len(Ket)))
     for i in range(len(Bra)):
        for j in range(len(Ket)):            
            S_m[i, j] = (Bra[i].dot(Ket[j])[0])
     return S_m


#SCF parameters
orbital_error = np.ones(len(spinorb_vec))

# SCF loop
while (max(orbital_error)) > prec:


    hdPhi_vec = [] 
    vPhi_vec = [] 
    alphaPhi_vec = []
    orb_energy_vec = []
    tmp2 = []

    for i in range(spinorb_vec.shape[0]):
        hdPhi_vec.append(orb.apply_dirac_hamiltonian(spinorb_vec[i], prec, 0.0))
        vPhi_vec.append(orb.apply_potential(-1.0, V_tree, spinorb_vec[i], prec))
        alphaPhi_vec.append(orb.gaunt(spinorb_vec[i],prec))
    hdPhi_vec = np.array(hdPhi_vec)
    vPhi_vec = np.array(vPhi_vec)
    alphaPhi_vec = np.array(alphaPhi_vec)

    JPhi_vec = J_n(spinorb_vec)
    KPhi_vec = K_n(spinorb_vec)
    GJPhi_vec = J_n(alphaPhi_vec)
    GKPhi_vec = K_n(alphaPhi_vec)

    
    # Definition of full 4c hamitoninan
    add_psi_vec = hdPhi_vec + vPhi_vec + JPhi_vec - KPhi_vec - GJPhi_vec + GKPhi_vec
    
    #Calculating of Fij with i /= j  and Lamda matrix
    Fij_matrix = calc_overlap(spinorb_vec, add_psi_vec)
    Lambda_n = np.diag(np.diag(Fij_matrix))
    

    orb_energy_vec = np.diag(Fij_matrix)
    print('Orb_Energy', orb_energy_vec)

    V_mat = calc_overlap(spinorb_vec, vPhi_vec)
    J_mat = calc_overlap(spinorb_vec, JPhi_vec)
    K_mat = calc_overlap(spinorb_vec, KPhi_vec)
    GJ_mat = calc_overlap(spinorb_vec, GJPhi_vec)
    GK_mat = calc_overlap(spinorb_vec, GKPhi_vec)

    E_orb  =  Fij_matrix.trace()
    E_en   =  V_mat.trace()
    E_coul =  0.5*J_mat.trace() - 0.5*GJ_mat.trace()
    E_ex   = -0.5*K_mat.trace() + 0.5*GK_mat.trace()
    E_tot  = E_orb - E_coul - E_ex 
    print("E_total(Dirac-Coulomb-Gaunt) approximiation", E_tot)

    
     
    # Calculation of necessary potential contributions to Hellmotz
    tmp = add_psi_vec + (Lambda_n - Fij_matrix) @ spinorb_vec
    
    for i in range(0, tmp.shape[0]): 
        tmp2.append(orb.apply_helmholtz(tmp[i], orb_energy_vec[i], c, prec))
    tmp2 =  np.array(tmp2)
    
    new_orbital_vec = []
    for i in range(spinorb_vec.shape[0]):
        new_orbital_vec.append(orb.apply_dirac_hamiltonian(tmp2[i], prec, orb_energy_vec[i]))
        new_orbital_vec[i].normalize()    
    new_orbital_vec = np.array(new_orbital_vec)



    delta_psi = new_orbital_vec - spinorb_vec
    orbital_error = np.array([delta_psi.norm() for delta_psi in delta_psi])


    # Compute overlap matrices
    S_tilde = calc_overlap(new_orbital_vec, new_orbital_vec)
    
    # Löwdin orthonormalization S^{-1/2} = U * Sigma^{-1/2} * U^T
    sigma, U = LA.eig(S_tilde)
    Sm5 = U @ np.diag(sigma**(-0.5)) @ U.transpose()
    spinorb_vec = Sm5 @  new_orbital_vec


hdPhi_vec = [] 
vPhi_vec = [] 
orb_energy_vec = []
tmp2 = []


for i in range(spinorb_vec.shape[0]):
    hdPhi_vec.append(orb.apply_dirac_hamiltonian(spinorb_vec[i], prec, 0.0))    
    vPhi_vec.append(orb.apply_potential(-1.0, V_tree, spinorb_vec[i], prec))
    alphaPhi_vec.append(orb.gaunt(spinorb_vec[i],prec))
hdPhi_vec = np.array(hdPhi_vec)
vPhi_vec = np.array(vPhi_vec)
alphaPhi_vec = np.array(alphaPhi_vec)

JPhi_vec = J_n(spinorb_vec)
KPhi_vec = K_n(spinorb_vec)
GJPhi_vec = J_n(alphaPhi_vec)
GKPhi_vec = K_n(alphaPhi_vec)

# Definition of full 4c hamitoninan
add_psi_vec = hdPhi_vec + vPhi_vec + JPhi_vec - KPhi_vec - GJPhi_vec + GKPhi_vec

#Calculating of Fij with i /= j  and Lamda matrix
Fij_matrix = calc_overlap(spinorb_vec, add_psi_vec)
Lambda_n = np.diag(np.diag(Fij_matrix))


orb_energy_vec = np.diag(Fij_matrix)
print('Orb_Energy', orb_energy_vec)

V_mat = calc_overlap(spinorb_vec, vPhi_vec)
J_mat = calc_overlap(spinorb_vec, JPhi_vec)
K_mat = calc_overlap(spinorb_vec, KPhi_vec)
GJ_mat = calc_overlap(spinorb_vec, GJPhi_vec)
GK_mat = calc_overlap(spinorb_vec, GKPhi_vec)

E_orb  =  Fij_matrix.trace()
E_en   =  V_mat.trace()
E_coul =  0.5*J_mat.trace() - 0.5*GJ_mat.trace()
E_ex   = -0.5*K_mat.trace() + 0.5*GK_mat.trace()
E_tot  = E_orb - E_coul - E_ex 
print("E_total(Dirac-Coulomb-Gaunt) approximiation", E_tot)